# Applied Data Science Capstone - Neighborhoods in Toronto: explore and clustering

In [95]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

#!conda install -c conda-forge geocoder 
#!pip install geocoder
import geocoder # import geocoder

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#!pip install folium
import folium

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import requests

I'll use pandas to read the wikipedia table using the method read_html and extracting the first table with css class 'wikitable'. (es: pandas.read_html(url, attrs={'class': 'wikitable'}))   

In [5]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data = pd.read_html(url, attrs={'class': 'wikitable'})   
df = pd.concat(data)

#3.1 The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
df.columns = ['PostalCode', 'Borough', 'Neighborhood']

#3.2 Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
#df = df[df['Borough'] != 'Not assigned']
df.drop(df[df['Borough']=="Not assigned"].index, axis=0, inplace=True)

#3.3  More than one neighborhood can exist in one postal code area. 
#     For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: 
#     Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
df = pd.DataFrame(df.groupby('PostalCode').agg(lambda x:', '.join(x.unique())))
df.reset_index(inplace=True)

#3.4 If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
#    So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
df.loc[df.Neighborhood == 'Not assigned', 'Neighborhood'] = df.Borough

df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [6]:
df.shape

(103, 3)

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

In [7]:
# The code was removed by Watson Studio for sharing.

In [8]:
def geocoder_google(postal_code, city= 'Toronto', country='Ontario'):
    
    # initialize your variable to None
    lat_lng_coords = None
    i = 0
    
    # loop until you get the coordinates
    while(lat_lng_coords is None and i < 3):
        address = '{}, {}, {}'.format(postal_code, city, country)
        g = geocoder.google(address, key=GOOGLE_API_KEY)
        lat_lng_coords = g.latlng
        i = i + 1
        #print(postal_code, lat_lng_coords)
    return lat_lng_coords

In [10]:
df['Coords'] = df.apply(lambda x: geocoder_google(x['PostalCode']), axis=1)
df['Latitude'] = df.apply(lambda x: x['Coords'][0], axis=1)
df['Longitude'] = df.apply(lambda x: x['Coords'][1], axis=1)
df.drop('Coords', axis=1, inplace=True)

df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [18]:
df.shape

(103, 5)

In [11]:
df.to_csv('toronto.csv', index=False)

In [ ]:
df1 = pd.read_csv('toronto.csv')

df1.head()

# Explore and cluster the neighborhoods in Toronto

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you. Just make sure:
   * to add enough Markdown cells to explain what you decided to do and to report any observations you make.
   * to generate maps to visualize your neighborhoods and how they cluster together. 

In [143]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: PHHF1GSKBJUGYWOKFIMGKJUFOIXBWEDQGYRNSGVSFGGH1BDS
CLIENT_SECRET:KH1DYCIEQMOANMNGZQJQN1XG0IDLWJT4O0NDX3ZMGZS03G2N


In [146]:
#filter the Borough that contain the word Toronto
df_toronto = df[df['Borough'].str.contains('Toronto')].sort_values('Borough')
df_toronto

def explore_venues_in_toronto(postal_codes, borough, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for postal_code, borough, lat, lng in zip(postal_codes, borough, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        #print(results[0]['venue'])
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            postal_code, 
            borough,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                             'Borough',
                             'PostalCode Latitude',
                             'PostalCode Longitude',
                             'Venue',
                             'Venue Latitude', 
                             'Venue Longitude',
                             'Venue Category']
    
    return(nearby_venues)

df_toronto.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
65,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678
64,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
49,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049
63,M5N,Central Toronto,Roselawn,43.711695,-79.416936
59,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752


In [148]:
df_venues = explore_venues_in_toronto(df_toronto['PostalCode'], df_toronto['Borough'], df_toronto['Latitude'], df_toronto['Longitude'])
df_venues[df_venues['Borough'] == 'Central Toronto'].head()

,PostalCode,Borough,PostalCode Latitude,PostalCode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M5R,Central Toronto,43.67271,-79.405678,Ezra's Pound,43.675153,-79.405858,Café
1,M5R,Central Toronto,43.67271,-79.405678,Roti Cuisine of India,43.674618,-79.408249,Indian Restaurant
2,M5R,Central Toronto,43.67271,-79.405678,Rose & Sons,43.675668,-79.403617,American Restaurant
3,M5R,Central Toronto,43.67271,-79.405678,Live Organic Food Bar,43.675053,-79.406715,Vegetarian / Vegan Restaurant
4,M5R,Central Toronto,43.67271,-79.405678,Fet Zun,43.675147,-79.406346,Middle Eastern Restaurant


In [180]:
print(df_venues.shape)
print('number of distinct Borough:', len(df_venues['Borough'].unique()))
df_venues.groupby('Venue Category').count()


(1714, 8)
number of distinct Borough: 4


,PostalCode,Borough,PostalCode Latitude,PostalCode Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,,
Afghan Restaurant,1,1,1,1,1,1,1
Airport,1,1,1,1,1,1,1
Airport Food Court,1,1,1,1,1,1,1
Airport Gate,1,1,1,1,1,1,1
Airport Lounge,2,2,2,2,2,2,2
Airport Service,2,2,2,2,2,2,2
Airport Terminal,2,2,2,2,2,2,2
American Restaurant,23,23,23,23,23,23,23
Antique Shop,2,2,2,2,2,2,2


In [179]:
map_borough = folium.Map(location=[df_toronto['Latitude'].mean(),
                                   df_toronto['Longitude'].mean()], 
                         zoom_start=12).add_to(folium.Figure(width=700, height=400))

mc = MarkerCluster()
rainbow = [colors.rgb2hex(i) for i in cm.rainbow(np.linspace(0, 1, len(df_venues['Borough'].unique()) ))]

#creating a Marker for each point in df_sample. Each point will get a popup with their zip
for (counter, row) in enumerate(df_venues.values):
    folium.CircleMarker(
        location=[row[2],  row[3]],
        radius=5,
        popup=row[4],
        color=rainbow[1],
        fill=True,
        fill_color=rainbow[2],
        fill_opacity=0.7).add_to(map_borough)

    map_borough.add_child(mc)

map_borough

In [113]:
#grab a random sample from df
fig = folium.Figure(width=500, height=200)

map_test = folium.Map(location=[df_toronto['Latitude'].mean(), 
                                df_toronto['Longitude'].mean()], 
                      zoom_start=10).add_to(folium.Figure(width=700, height=400))

#creating a Marker for each point in df_sample. Each point will get a popup with their zip
for row in df_toronto.itertuples(): 
    map_test.add_child(
        folium.Marker(
            location=[row.Latitude ,row.Longitude],
            popup=row.Borough))

map_test

In [114]:
map_borough = folium.Map(location=[df_toronto['Latitude'].mean(),
                                   df_toronto['Longitude'].mean()], 
                         zoom_start=12).add_to(folium.Figure(width=700, height=400))

mc = MarkerCluster()
rainbow = [colors.rgb2hex(i) for i in cm.rainbow(np.linspace(0, 1, len(df_toronto)))]

#creating a Marker for each point in df_sample. Each point will get a popup with their zip
for (counter, row) in enumerate(df_toronto.values):
    folium.CircleMarker(
        location=[row[3],  row[4]],
        radius=5,
        popup=row[1],
        color=rainbow[counter],
        fill=True,
        fill_color=rainbow[counter],
        fill_opacity=0.7).add_to(map_borough)

    map_borough.add_child(mc)

map_borough

In [68]:
df[df['Borough'].str.contains('Toronto')]

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
49,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


In [168]:
df_venues.values


array([['M5R', 'Central Toronto', 43.6727097, ..., 43.67515283323029,
        -79.40585846415303, 'Café'],
       ['M5R', 'Central Toronto', 43.6727097, ..., 43.67461834990478,
        -79.40824866273744, 'Indian Restaurant'],
       ['M5R', 'Central Toronto', 43.6727097, ..., 43.6756684758828,
        -79.40361659752443, 'American Restaurant'],
       ...,
       ['M6S', 'West Toronto', 43.6515706, ..., 43.650675, -79.479368,
        'Spa'],
       ['M6S', 'West Toronto', 43.6515706, ..., 43.64796649019104,
        -79.48646216806426, 'Gym'],
       ['M6S', 'West Toronto', 43.6515706, ..., 43.65516029673073,
        -79.48697089772627, 'Restaurant']], dtype=object)